# Pré-processamento: Parte I

## Importar os pacotes que serão usados

In [1]:
import os
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import GA_ReadOnly
from multiprocessing import Pool, Manager, cpu_count
from osgeo_utils.gdal_calc import *

ERROR 1: libopenjp2.so.7: cannot open shared object file: No such file or directory
ERROR 1: libopenjp2.so.7: cannot open shared object file: No such file or directory
ERROR 1: libwebp.so.7: cannot open shared object file: No such file or directory
ERROR 1: libwebp.so.7: cannot open shared object file: No such file or directory
ERROR 1: libmariadb.so.3: cannot open shared object file: No such file or directory
ERROR 1: libmariadb.so.3: cannot open shared object file: No such file or directory
ERROR 1: libjxl.so.0.8: cannot open shared object file: No such file or directory
ERROR 1: libjxl.so.0.8: cannot open shared object file: No such file or directory
ERROR 1: libOpenEXR-3_1.so.30: cannot open shared object file: No such file or directory
ERROR 1: libOpenEXR-3_1.so.30: cannot open shared object file: No such file or directory
ERROR 1: libhdf5.so.310: cannot open shared object file: No such file or directory
ERROR 1: libhdf5.so.310: cannot open shared object file: No such file or dire

## Checagem de arquivos

Fazer uma checagem dos arquivos que serão lidos, deve conter todos os biomas que foram rasterizados do Inventário Nacional no formato inventario_*bioma*_byte.tif.

In [2]:
# Especifica o caminho da pasta que contém os arquivos
pasta = '../dados/inventario_v4_2016/inventario_biomas_rasters'

# Obtém a lista de arquivos na pasta
arquivos = os.listdir(pasta)

for arquivo in arquivos:
    print(arquivo)

inventario_cerrado_byte.tif
inventario_caatinga_byte.tif
inventario_mata_atlantica_byte.tif
inventario_pantanal_byte.tif
inventario_pampa_byte.tif
inventario_amazonia_byte.tif


## Criar as máscaras

O código abaixo transfoma os mapas rasterizados do inventário em uma máscara binária, 1 onde o valor de pixel é maior que zero e 0 para o restante.
Isso serve para criar o contorno do bioma que será replicado no MapBiomas, garantindo que os mapas terão a mesma quantidade de linha e colunas para serem comparáveis.

In [5]:
# Supress the warning
np.seterr(divide='ignore', invalid='ignore')

# Função que processa um arquivo
def process_file(arquivo, pasta_inventario, pasta_biomas_mask, n_processo, status):
    print(f"Processo {n_processo}: processando o arquivo {arquivo}...\n")

    Calc("((A > 0)/(A > 0))", A=f"{pasta_inventario}/{arquivo}", 
         outfile=f"{pasta_biomas_mask}/mask{arquivo.split('inventario')[1]}",
         type='Byte', overwrite=True, extent='intersect', quiet=True)
    
    print(f"Processo {n_processo}: o arquivo {arquivo} foi processado com sucesso!\n")
    status[arquivo] = False


if __name__ == '__main__':
    # Especifica o caminho da pasta que contém os arquivos
    pasta_inventario = '../dados/inventario_v4_2016/inventario_biomas_rasters'
    pasta_biomas_mask = '../dados/biomas_mask'

    # Obtém a lista de arquivos na pasta
    arquivos = os.listdir(pasta_inventario)

    # Define o número máximo de processos em execução simultaneamente
    max_processes = 2

    # Inicializa o pool de processos e o objeto Manager para compartilhar dados entre processos
    pool = Pool(processes=max_processes)
    manager = Manager()

    # Cria um dicionário compartilhado para armazenar o status de processamento de cada arquivo
    status = manager.dict([(arquivo, False) for arquivo in arquivos])

    # Loop pelos arquivos e executa o mesmo processo em cada um, usando o pool de processos
    for i, arquivo in enumerate(arquivos):
        # Aguarda até que haja um processo livre para executar o próximo arquivo
        while True:
            if sum(status.values()) < max_processes:
                break

        # Define o status do arquivo como "em processamento"
        status[arquivo] = True

        # Processa o arquivo em um novo processo
        pool.apply_async(process_file, args=(arquivo, pasta_inventario, pasta_biomas_mask, i, status))

    # Aguarda todos os processos terminarem
    pool.close()
    pool.join()

    # Verifica se todos os processos foram finalizados
    if sum(status.values()) == 0:
        print("Todos os arquivos foram processados com sucesso!")

Processo 1: processando o arquivo inventario_caatinga_byte.tif...

Processo 0: processando o arquivo inventario_cerrado_byte.tif...

Processo 1: o arquivo inventario_caatinga_byte.tif foi processado com sucesso!

Processo 2: processando o arquivo inventario_mata_atlantica_byte.tif...

Processo 0: o arquivo inventario_cerrado_byte.tif foi processado com sucesso!

Processo 3: processando o arquivo inventario_pantanal_byte.tif...

Processo 3: o arquivo inventario_pantanal_byte.tif foi processado com sucesso!

Processo 4: processando o arquivo inventario_pampa_byte.tif...

Processo 4: o arquivo inventario_pampa_byte.tif foi processado com sucesso!

Processo 5: processando o arquivo inventario_amazonia_byte.tif...

Processo 2: o arquivo inventario_mata_atlantica_byte.tif foi processado com sucesso!

Processo 5: o arquivo inventario_amazonia_byte.tif foi processado com sucesso!

Todos os arquivos foram processados com sucesso!
